## Particle Swarm Optimization (PSO)
PSO는 SVM 매개변수 조정이나 퍼지 클러스터링에서 사용되며, 빠르고 안정적인 성능을 보임

## 단일 목적
### 목적 1. price 최소화/최대화

In [66]:
import pandas as pd
import numpy as np
from pyswarm import pso
from joblib import load

# 1. Surrogate 모델 불러오기 (pkl 파일)
surrogate_model = load('/data/ephemeral/home/Sojeong/level4-cv-finalproject-hackathon-cv-02-lv3/Sojeong/surrogate/autosklearn_model.pkl')

# # 최적 모델 가져오기
# models_with_weights = surrogate_model.get_models_with_weights()
# best_model = models_with_weights[0][1]  # 최적 모델만 추출 (파이프라인 포함)

# 2. 데이터셋 불러오기
df = pd.read_csv("/data/ephemeral/home/Sojeong/level4-cv-finalproject-hackathon-cv-02-lv3/Sojeong/surrogate/melb_split.csv")
data = df.drop(['Price', 'Split', 'Address', 'BuildingArea', 'YearBuilt'], axis=1)  # 제외한 데이터

# 3. 목표 함수 정의
def objective_function(x):
    """
    PSO의 목적 함수
    x: 최적화 대상 변수 리스트 (['Distance', 'Landsize'])
    return: 목표값과 예측값(Predicted) 간의 차이
    """
    #target_price = 10000000  # 최적화 목표값
    
    # Distance와 Landsize 열을 업데이트
    data_copy = data.copy()
    data_copy["Distance"] = x[0]
    data_copy["Landsize"] = x[1]
    print(x[0], x[1])

    # Price 열 제거 (모델 학습 시 타겟 열)
    input_features = data_copy#.drop(columns=["Price"])

    # Surrogate 모델로 예측 수행
    predicted_prices = surrogate_model.predict(input_features)

    # 모든 예측 값과 목표값의 평균 차이 반환
    #return np.mean(np.abs(predicted_prices - target_price)) # y_hat - y(Price값이 아니라 우리가 원하는 목표값)
    #return np.mean(predicted_prices)  # Price값을 최소화
    return -np.mean(predicted_prices)  # Price값을 최대화

# 4. PSO 최적화
# 각 변수의 범위 설정
lb = [0, 0]  # 'Distance', 'Landsize' 최소값
ub = [38, 4583]  # 'Distance', 'Landsize' 최대값

# PSO 실행
best_x, best_f = pso(objective_function, lb, ub, swarmsize=30, maxiter=25, debug=True)

# 결과 출력
print("최적화된 특성 값 ('Distance', 'Landsize'):", best_x)
print("목표값과의 차이:", best_f)

No constraints given.
28.06987461336413 2975.4722432317835
17.06590586709156 2793.3796590967436
23.884227963574617 23.006065731755548
6.834400725179767 2373.7197322891966
19.15469841228908 202.4606080749259
26.975100215033798 151.86086106068652
25.22724996571385 1811.9430916922265
23.53750735054615 3738.729418214659
23.629063210540522 3630.3567005443897
9.458273913847494 1406.833157285049
6.851928253016253 23.03215801011429
35.60445879214568 222.70191222487156
18.23314815447858 8.846171080891473
4.235457532941757 2808.5631421897674
17.37284862464834 2707.8246655806524
37.81407323113411 4126.569127310601
1.50439679543737 465.8699552850533
29.711479973572622 4471.357504926605
32.30923928390201 1891.0154243390518
12.142009164781188 183.5984379568664
16.82167810663556 4530.220429185286
24.920784311775638 3551.8755098972765
17.296754574697392 2474.463490532759
24.412860947493456 430.71453409693396
12.326804706287945 1632.2662512523916
29.012206187655536 3407.556198958696
23.97181396480639 6

KeyboardInterrupt: 

In [58]:
# 최적화된 결과를 Surrogate 모델로 확인
# Distance와 Landsize 열을 업데이트
data_copy1 = data.copy()
data_copy1["Distance"] = best_x[0]
data_copy1["Landsize"] = best_x[1]

# Price 열 제거 (모델 학습 시 타겟 열)
input_features1 = data_copy1#.drop(columns=["Price"])

# Surrogate 모델로 예측 수행
final_predicted_price = surrogate_model.predict(input_features1)
print(final_predicted_price.shape)    
print("최적화된 Predicted 값:", final_predicted_price)

(13580,)
최적화된 Predicted 값: [1788829.30078125  894908.484375   1109779.00390625 ...  955726.54492188
 1457456.4765625  1029908.5703125 ]


In [59]:
best_x

array([  10.        , 1486.99997359])

## 2. multi-objective 
### 목적 1. Price 최소화/최대화
### 목적 2.  Distance 최소화

- 방법 1 - 가중합
- 방법 2 - 파레토 기반

### 방법 1 - 가중합
(가중치를 내가 직접 줘야함)

In [64]:
# 다중 목표 함수 정의
def multi_objective_function(x, w1=0.7, w2=0.3):
    """
    PSO의 다중 목표 함수
    x: 최적화 대상 변수 리스트 (['Distance', 'Landsize'])
    w1: 첫 번째 목표의 가중치 (default: 0.7)
    w2: 두 번째 목표의 가중치 (default: 0.3)
    return: 가중합으로 변환된 단일 목표 값
    """
    # Distance와 Landsize 열을 업데이트
    data_copy = data.copy()
    data_copy["Distance"] = x[0]
    data_copy["Landsize"] = x[1]

    # Surrogate 모델로 예측 수행
    input_features = data_copy
    predicted_prices = surrogate_model.predict(input_features)

    # 목표 1: Price와 목표값 차이
    objective1 = np.mean(predicted_prices) # 최소화
    #objective1 = -np.mean(predicted_prices) # 최대화

    # 목표 2: Distance 최소화
    objective2 = x[0]

    # 가중합 계산
    return w1 * objective1 + w2 * objective2

# PSO 실행
# lb와 ub는 'Distance'와 'Landsize'의 탐색 범위
best_x, best_f = pso(multi_objective_function, lb, ub, args=(0.7, 0.3), swarmsize=30, maxiter=5, debug=True)

# 결과 출력
print("최적화된 특성 값 ('Distance', 'Landsize'):", best_x)
print("가중합된 다중 목표 값:", best_f)

No constraints given.
New best for swarm at iteration 1: [21.64673097  0.        ] 462483.1477213193
New best for swarm at iteration 1: [25.07367246  0.        ] 429005.53228821576
New best for swarm at iteration 1: [32.80859153  0.        ] 391873.71813271515
Best after iteration 1: [32.80859153  0.        ] 391873.71813271515
New best for swarm at iteration 2: [38.  0.] 373982.56353387266
Best after iteration 2: [38.  0.] 373982.56353387266
Best after iteration 3: [38.  0.] 373982.56353387266
Best after iteration 4: [38.  0.] 373982.56353387266
Best after iteration 5: [38.  0.] 373982.56353387266
Stopping search: maximum iterations reached --> 5
최적화된 특성 값 ('Distance', 'Landsize'): [38.  0.]
가중합된 다중 목표 값: 373982.56353387266


### 방법 2. 파라토 기반

In [65]:
# 파레토 우월성 비교 함수
def is_dominated(f1, f2):
    return all(f1 <= f2) and any(f1 < f2)  # f1이 f2를 우월하면 True 반환

# 다중 목표 PSO 함수 정의
def pso_pareto(funcs, lb, ub, swarmsize=30, maxiter=5):
    """
    다중 목표 PSO (파레토 최적화 기반)
    funcs: 다중 목표 함수 리스트 [func1, func2, ...]
    lb: 하한 리스트
    ub: 상한 리스트
    """
    S = swarmsize
    D = len(lb)
    x = np.random.rand(S, D) * (ub - lb) + lb  # 초기 위치
    v = np.zeros_like(x)
    personal_best = x.copy()
    personal_fitness = [np.inf] * S
    global_pareto = []

    for iteration in range(maxiter):
        fitness = np.array([[func(xi) for func in funcs] for xi in x])  # 다중 목표 값 계산

        # 개인 최적 위치 갱신
        for i in range(S):
            if is_dominated(fitness[i], personal_fitness[i]):
                personal_best[i] = x[i]
                personal_fitness[i] = fitness[i]

        # 전역 파레토 최적 집합 갱신
        for f, xi in zip(fitness, x):
            dominated = False
            for fg in global_pareto:
                if is_dominated(f, fg):
                    dominated = True
                    break
            if not dominated:
                global_pareto.append(f)

        # 입자 속도와 위치 업데이트
        rp = np.random.rand(S, D)
        rg = np.random.rand(S, D)
        v = 0.5 * v + 0.5 * rp * (personal_best - x)
        x = x + v

    return global_pareto


# 다중 목표 함수 정의
def objective1(x):
    data_copy = data.copy()
    data_copy["Distance"] = x[0]
    data_copy["Landsize"] = x[1]
    predicted_prices = surrogate_model.predict(data_copy)
    return np.mean(predicted_prices)  # Price 최소화

def objective2(x):
    return x[0]  # Distance 최소화

# 실행
pareto_front = pso_pareto([objective1, objective2], lb, ub)

# 결과 출력
print("파레토 최적 집합:", pareto_front)


TypeError: unsupported operand type(s) for -: 'list' and 'list'